In [1]:
import numpy as np 
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from nltk import word_tokenize


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# All Rock
rock1 = open('/content/drive/My Drive/Colab Notebooks/AllRock.txt', 'r').read()
rock = ''.join([i for i in rock1 if not i.isdigit()]).replace("\n", " ").lower().replace(".", "").replace("!", "").replace("?", "").replace(",", "").replace("", "").replace("'", "").replace(")", "").replace("(", "").split(' ')
# All Pop
pop1 = open('/content/drive/My Drive/Colab Notebooks/AllPop.txt', 'r').read()
pop = ''.join([i for i in pop1 if not i.isdigit()]).replace("\n", " ").lower().replace(".", "").replace("!", "").replace("?", "").replace(",", "").replace("", "").replace("'", "").replace(")", "").replace("(", "").split(' ')
# All Country
country1 = open('/content/drive/My Drive/Colab Notebooks/AllCountry.txt', 'r').read()
country = ''.join([i for i in country1 if not i.isdigit()]).replace("\n", " ").lower().replace(".", "").replace("!", "").replace("?", "").replace(",", "").replace("", "").replace("'", "").replace(")", "").replace("(", "").split(' ')
# All Rap
rap1 = open('/content/drive/My Drive/Colab Notebooks/AllLyrics.txt', 'r').read()
rap = ''.join([i for i in rap1 if not i.isdigit()]).replace("\n", " ").lower().replace(".", "").replace("!", "").replace("?", "").replace(",", "").replace("", "").replace("'", "").replace(")", "").replace("(", "").split(' ')

In [5]:
SONG_LENGTH = 200
# Rock
Rock = [rock[i*SONG_LENGTH:(i+1)*SONG_LENGTH] for i in range(0,int(len(rock)/200))]
# Country
Country = [country[i*SONG_LENGTH:(i+1)*SONG_LENGTH] for i in range(0,int(len(country)/200))]
# Pop
Pop = [pop[i*SONG_LENGTH:(i+1)*SONG_LENGTH] for i in range(0,int(len(pop)/200))]
# Rap
Rap = [rap[i*SONG_LENGTH:(i+1)*SONG_LENGTH] for i in range(0,int(len(rap)/200))]

In [6]:
ds_rock = [' '.join(Rock[i]) for i in range(len(Rock))]
ds_country = [' '.join(Country[i]) for i in range(len(Country))]
ds_pop = [' '.join(Pop[i]) for i in range(len(Pop))]
ds_rap = [' '.join(Rap[i]) for i in range(len(Rap))]

In [7]:
# rock 0, country 1, pop 2, rap 3
ds_ro = []
genre = 0
for sample in ds_rock:
  ds_ro.append([genre, sample])

ds_co = []
genre = 1
for sample in ds_country:
  ds_co.append([genre, sample])

ds_po = []
genre = 2
for sample in ds_pop:
  ds_po.append([genre, sample])

ds_ra = []
genre = 3
for sample in ds_rap:
  ds_ra.append([genre, sample])

ds = ds_ro+ds_co+ds_po+ds_ra

ds = np.array(ds)
print('Genres: ', ds[:, 0])
print('Lyrics: ', ds[:, 1])

Genres:  ['0' '0' '0' ... '3' '3' '3']
Lyrics:  ['yesterday all my troubles seemed so far away now it looks as though theyre here to stay oh i believe in yesterday suddenly im not half the man i used to be theres a shadow hanging over me oh yesterday came suddenly why she had to go i dont know she wouldnt say i said something wrong now i long for yesterday yesterday love was such an easy game to play now i need a place to hide away oh i believe in yesterday why she had to go i dont know she wouldnt say i said something wrong now i long for yesterday yesterday love was such an easy game to play now i need a place to hide away oh i believe in yesterday mm mm mm mm mm mm mm when i find myself in times of trouble mother mary comes to me speaking words of wisdom let it be and in my hour of darkness she is standing right in front of me speaking words of wisdom let it be let it be let it be let it be let it be whisper words of wisdom let it be and when the broken hearted people living'
 'in t

In [8]:
# max words we use
maxwords = 50000
# max words per lyric (should be 200 anyway as set up)
maxlyricwords = 200
# tokenize here
embedding_dimension = 100
tokenizer = Tokenizer(num_words=maxwords, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(ds[:, 1])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 16903 unique tokens.


In [9]:
X = tokenizer.texts_to_sequences(ds[:, 1])
X = pad_sequences(X, maxlen=maxlyricwords)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (2625, 200)


In [10]:
Y = pd.get_dummies(ds[:, 0])
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (2625, 4)


In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2362, 200) (2362, 4)
(263, 200) (263, 4)


In [12]:
model = Sequential()
model.add(Embedding(maxwords, embedding_dimension, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 10
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/10
34/34 [==============================] - 26s 670ms/step - loss: 1.2689 - accuracy: 0.5284 - val_loss: 0.9728 - val_accuracy: 0.6582
Epoch 2/10
34/34 [==============================] - 22s 644ms/step - loss: 0.9991 - accuracy: 0.6092 - val_loss: 0.7254 - val_accuracy: 0.6920
Epoch 3/10
34/34 [==============================] - 22s 646ms/step - loss: 0.7042 - accuracy: 0.7119 - val_loss: 0.5453 - val_accuracy: 0.7975
Epoch 4/10
34/34 [==============================] - 22s 639ms/step - loss: 0.5501 - accuracy: 0.7776 - val_loss: 0.5756 - val_accuracy: 0.7806
Epoch 5/10
34/34 [==============================] - 22s 644ms/step - loss: 0.4478 - accuracy: 0.8166 - val_loss: 0.5037 - val_accuracy: 0.7848
Epoch 6/10
34/34 [==============================] - 22s 645ms/step - loss: 0.3402 - accuracy: 0.8811 - val_loss: 0.5195 - val_accuracy: 0.8312
Epoch 7/10
34/34 [==============================] - 22s 636ms/step - loss: 0.2877 - accuracy: 0.9161 - val_loss: 0.5432 - val_accuracy: 0.8017

In [13]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

9/9 [==============================] - 1s 55ms/step - loss: 0.8007 - accuracy: 0.6806
Test set
  Loss: 0.801
  Accuracy: 0.681


In [14]:
lyric = ['Bumpin i meant for you call my ninja like']
seq = tokenizer.texts_to_sequences(lyric)
padded = pad_sequences(seq, maxlen=maxlyricwords)
pred = model.predict(padded)
labels = ['rock', 'country', 'pop', 'rap']
print(pred, labels[np.argmax(pred)])

[[9.3246537e-04 2.2743614e-03 9.0647396e-03 9.8772848e-01]] rap


In [15]:
def classify_string(input):
  lyric = [str(input)]
  seq = tokenizer.texts_to_sequences(lyric)
  padded = pad_sequences(seq, maxlen=maxlyricwords)
  pred = model.predict(padded)
  labels = ['rock', 'country', 'pop', 'rap']
  print(pred, labels[np.argmax(pred)])

In [16]:
def classify_list(input):
  for lyric in input:
    classify_string(lyric)

In [17]:
# markov generated lyrics
lyrics = ['Bumpin i meant for you call my ninja like',
 'Biz dont take their baby mommas ninja frick you nasty boy you',
 'Shifty sticks and pray and flee the frick all of you',
 'Glocks but all ill die slow',
 'Wondering if im askin blunt sip champagne range rover been outside for',
 'And youre so take that crown two pounds you know',
 'Publishing i thought i get witcha can i could cop',
 'Miss the more cause you in the right one',
 'Onyx and them hoes i love',
 'Gat call me puff daddy biggie gots ta like',
 'Everything around me shit b***** in ya imma stay yappin when',
 'Hum all about fingers in the loot im',
 'Rollem up heard whos this yeah keep on top sky is',
 'Drunk of ninjaz from now drop to',
 'Declinin windin like flypaper neighbor slow down',
 'Expensive cars i tote my crew i only got enough heart',
 'Lame dudes whos next move but the drugs to spit phrases thatll',
 'Guy well its cool and your poop so hard to',
 'Clap wit my life in ma little nasty boy',
 'Dial you should too much better man played',
 'Lali like that you frick doin all mcs have']

classify_list(lyrics)

[[9.3246537e-04 2.2743614e-03 9.0647396e-03 9.8772848e-01]] rap
[[5.3449156e-04 1.0384670e-03 3.6795407e-03 9.9474746e-01]] rap
[[0.00694316 0.01209779 0.02896835 0.9519907 ]] rap
[[0.0267585  0.03472553 0.21301189 0.7255041 ]] rap
[[0.00766694 0.01319862 0.03287706 0.9462574 ]] rap
[[0.0049939  0.00974889 0.03715449 0.9481027 ]] rap
[[0.01093788 0.0181043  0.03528805 0.9356698 ]] rap
[[0.04517562 0.05585761 0.17473097 0.72423583]] rap
[[0.02945629 0.046281   0.1403878  0.78387487]] rap
[[3.4327971e-04 7.7771040e-04 2.4827174e-03 9.9639636e-01]] rap
[[0.0101559  0.01481005 0.04598096 0.92905307]] rap
[[0.06712554 0.08645029 0.2942437  0.5521804 ]] rap
[[0.00681905 0.00992719 0.03780306 0.9454507 ]] rap
[[0.00834556 0.01461661 0.03983233 0.93720555]] rap
[[0.05570104 0.06263188 0.26916897 0.61249804]] rap
[[0.05317727 0.05466438 0.2998549  0.59230345]] rap
[[0.00207806 0.00393879 0.00992926 0.9840539 ]] rap
[[0.01406388 0.02029625 0.06429743 0.9013425 ]] rap
[[0.02922251 0.03943561 0.14

In [18]:
# lstm generated lyrics
lyrics2 = ['in the veins hard to explain how i maintain', 
  'to put my back in the house so i can i wanna flaunt you thats right', 
  'with the grime of my ninja frick',
  'with the ds crept in blastin him you dont want to slit the clits alot',
  'used to lick the clits a lot of problems never be the beamer with the goldie sound',
  'like a steelo not my steelo oh no thats not my my steelo oh i steelo not my steelo oh no',
  'thats not my no steelo bust my no dough day but this sittin bodies not my']

classify_list(lyrics2)

[[0.02854315 0.03963362 0.19643253 0.73539066]] rap
[[0.00996829 0.0166299  0.03103369 0.94236815]] rap
[[8.0073701e-04 1.9726395e-03 1.2132680e-02 9.8509395e-01]] rap
[[0.00957072 0.016364   0.02575561 0.94830966]] rap
[[0.00458604 0.00808106 0.0102652  0.9770677 ]] rap
[[0.05101185 0.05620176 0.1531064  0.73967993]] rap
[[0.00908127 0.01308981 0.04034968 0.9374792 ]] rap
